In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    # data.TensorDataset(*data_arrays) 将传入的 data_arrays 中的多个张量（如特征和标签）组合成一个 PyTorch 的 TensorDataset 对象。*data_arrays 是解包操作符，它可以将 list 或 tuple 元素解包成函数参数。
    dataset = data.TensorDataset(*data_arrays)
    # data.DataLoader 是 PyTorch 中的一个数据加载器，用于按批次加载数据。
    # shuffle=is_train 如果是训练集 (is_train=True)，则会对数据进行打乱（shuffle=True），这样可以防止模型学习到数据的顺序。如果是验证集或测试集，通常不打乱数据。
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[-0.5967,  0.7922],
         [-0.1097, -1.0611],
         [ 0.0799,  1.0233],
         [-0.0200, -0.1741],
         [ 0.6622, -2.1335],
         [-0.9626,  1.8821],
         [-1.0281,  1.1155],
         [ 0.2602, -1.9458],
         [-0.9818,  0.7919],
         [-0.0602, -0.0865]]),
 tensor([[ 0.3087],
         [ 7.5849],
         [ 0.8756],
         [ 4.7293],
         [12.7844],
         [-4.1238],
         [-1.6422],
         [11.3344],
         [-0.4484],
         [ 4.3572]])]

In [ ]:
# nn是神经网络的缩写
from torch import nn
# 在PyTorch中，全连接层在Linear类中定义。 第一个参数指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。
net = nn.Sequential(nn.Linear(2, 1))

In [5]:
# 通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。还可以使用替换方法normal_和fill_来重写参数值
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [6]:
# PyTorch还提供了均方误差损失函数，它实例化为一个对象。 从nn模块导入MSELoss类，该类也称为平方L2范数。 默认情况下，它返回所有样本损失的平均值。
loss = nn.MSELoss()
# torch.optim.SGD 用于实现 随机梯度下降的优化器类。
# net.parameters() 获取神经网络模型 net 的所有可学习参数（即权重和偏置）。这通常是一个可迭代的对象，包含了网络中的所有参数。
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        # 在每次迭代前，清除模型参数的梯度。
        trainer.zero_grad()
        l.backward()
        # 调用优化器的step函数来迭代模型参数。
        trainer.step()
    # 每个epoch结束后，打印损失值
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000279
epoch 2, loss 0.000097
epoch 3, loss 0.000097


In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0007, -0.0003])
b的估计误差： tensor([0.0006])


1.如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
若使用总损失，则学习率通常设置为 lr。
若使用平均损失，则可以将学习率 lr 增大为 lr * N（这里 N 是批量大小），以补偿梯度的缩小。

2.如何访问线性回归的梯度？
在反向传播后，你可以通过 w.grad 和 b.grad 来访问参数 w 和 b 的梯度。